In [ ]:
import sys
sys.path.append('../')
from gymenv_v2 import make_multiple_env
import json
easy_config = {
    "load_dir"        : '../instances/train_10_n60_m60',
    "idx_list"        : list(range(10)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
    }
hard_config = {
    "load_dir"        : '../instances/train_100_n60_m60',
    "idx_list"        : list(range(99)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}
test_config = {
"load_dir" : '../instances/test_100_n60_m60',
"idx_list" : list(range(99)),
"timelimit" : 50,
"reward_type" : 'obj'
}

In [ ]:
from src.seq2seq_policyNet.policy_roller import Observer as cnn_attention_observer
from src.seq2seq_policyNet.policy_roller import Agent as cnn_attention_agent

In [ ]:
import torch
from torch.distributions import Categorical
from src.seq2seq_policyNet.policy import Policy
from src.cnn_seseq2seq_policyNetq2seq.policy_roller import Observer

In [ ]:
Categorical(torch.FloatTensor([.2, .8]))

In [ ]:
class Agent(object):
    def __init__(self, observer, model):
        self.policy = model.eval()
        self.observer=observer
        self.rewards=[]
    def select_action(self, state, prev):
        probs, prev = self.policy(state, prev)
        action = probs.argmax()
        return action.item(), prev
    
    def report_reward(self, reward):
        self.rewards.append(reward)
    def run_episode(self):
        self.observer.run_episode(self)
    def finish_episode(self):
        rewards = self.rewards
        reward = sum(rewards)
        self.rewards = []
        return reward

In [ ]:
models = json.load(open('/seq2seq.json'))

In [ ]:
with open('seq2seq_results.json', 'w') as f:
    json.dump({}, f)
from tqdm.notebook import tqdm

In [ ]:
for model_id in tqdm(models):
    res = json.load(open('seq2seq_results.json'))
    if model_id not in res.keys():
        model = Policy(60, [32,64, 32], 
                        models[model_id]
                        )
        model.load_state_dict(torch.load(f'../trained_models/{model_id}'))
        result = {}
        for env_config, env_name in zip([easy_config, hard_config, test_config], \
                                        ['easy_config', 'hard_config', 'test_config']):
            observer = Observer(env_config)
            agent = Agent(observer, model)
            rewards = []
            for iteration in range(12):
                agent.run_episode()
                reward = agent.finish_episode()
                rewards.append(reward)
            rewards = sorted(rewards)[1:-1]
            reward = sum(rewards)/len(rewards)
            result[env_name] = reward
        res[model_id] = result
    with open('seq2seq_results.json', 'w') as f:
        json.dump(res, f)